In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys 
import time 

liens_publications =[]

descriptions=[]

reactions = []

likes =[]
loves =[]
angryy =[]

produits=[]


def extract_publications_links():
    publications = driver.find_elements_by_tag_name('article')
    i=1
    for pub in publications:
        a = pub.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div/section/article['+str(i)+']/footer/div[2]/a[1]')
        #print(a.get_attribute('href'))
        liens_publications.append(a.get_attribute('href'))
        i=i+1

    
    return(liens_publications)

def extract_description(lien):
    driver.get(lien)
    description = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div[1]/div/p')
    return(description.text)


def produit_par_publication(pub):
    res=""
    product_list=["carte","بطاقة بنكية","double salire","credit","Crédithabitat","crédit" ,"9ardh","9arth","cartebancaire","DigiCarte","STBNet"]
    for p in product_list:
        if(type(pub)==list):
            break
        if(pub.find(p)!=-1):
            if(p=="بطاقة بنكية"): 
                res="Carte Banquaire"
            break
        else:
            res="Neutre"
    return(res)

def nombre_reactions():
    i=1
    while(i<6):
        react = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div/section/article['+str(i)+']/footer/div[2]/span[1]/a[1]').text
        i=i+1
        #print(react)
        reactions.append(react)
    return(reactions)


driver = webdriver.Chrome('/Users/Dream/Desktop/Test/chromedriver')
driver.get('https://mbasic.facebook.com')


# Step 3) Search & Enter the Email or Phone field & Enter Password
username = driver.find_element_by_id("m_login_email")
password = driver.find_element_by_name("pass")
submit   = driver.find_element_by_name("login")
username.send_keys("")
password.send_keys("")
# Step 4) Click Login
submit.click()
time.sleep(10)

confirm = driver.find_element_by_xpath('/html/body/div/div/div/div/table/tbody/tr/td/div/form/div/input')
confirm.click()
time.sleep(10)

search = driver.find_element_by_xpath('/html/body/div/div/div[1]/div/form/table/tbody/tr/td[2]/input')
search.send_keys("SocieteTunisiennedeBanque")
search.send_keys(Keys.ENTER)
time.sleep(10)




In [7]:
stb = driver.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div/div/section/div/div/div/div/article/div/header/table/tbody/tr/td[2]/header/h3/span/strong/a')
stb.click()
time.sleep(10)

reactions = nombre_reactions()
time.sleep(10)

extract_publications_links()

for lien in liens_publications:
    desc=extract_description(lien)
    descriptions.append(desc)

for d in descriptions:
    produit =  produit_par_publication(d)
    produits.append(produit)


    
for lien in liens_publications:
    driver.get(lien)
    react = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[3]/a').get_attribute('href')
    time.sleep(10)
    driver.get(react)
    like = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[2]/span').text
    love = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[3]/span').text
    try:
        angry = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[4]/img').get_attribute('alt')
        if (angry=='Angry'):
            angry = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[4]/span').text
        else:
            angry='0'
    except:
        angry='0'
    likes.append(like)
    loves.append(love)
    angryy.append(angry)




In [9]:
import pandas as pd
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'links': liens_publications,'description':descriptions, 'produit': produits ,'reactions': reactions ,'Like' : likes , 'love' : loves ,'Angry':angryy})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple4.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [4]:
from selenium.common.exceptions import NoSuchElementException
def extract_commentaires():
    err=''
    i=1
    comments=[]
    done=False
    try:
        while(True):
                try:
                    commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                    i=i+1
                    #print(commentaire.text)
                    comments.append(commentaire.text)
                except:
                    while(True):
                        page = driver.find_element_by_link_text('View more comments…')
                        page.click()
                        #print('ok')
                        j=10
                        try:
                            i=2
                            while(True):
                                commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                                i=i+1
                                #print(commentaire.text)
                                comments.append(commentaire.text)
                                j=j+10

                        except:
                            while(True):
                                try:
                                        page = driver.find_element_by_link_text('View more comments…')
                                        page.click()
                                        #print('ok')
                                        i=2
                                        while(True):
                                            commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                                            i=i+1
                                            #print(commentaire.text)    
                                            comments.append(commentaire.text)
                                        j=j+10
                                except NoSuchElementException:
                                    done=True
                                    break

    except:
                    err='erreur'
    return(comments)


users=[]
def extract_user_names():
    err=''
    i=1
    comments=[]
    done=False
    try:
        while(True):
                try:
                    commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                    i=i+1
                    #print(commentaire.text)
                    users.append(commentaire.text)
                except:
                    while(True):
                        page = driver.find_element_by_link_text('View more comments…')
                        page.click()
                        #print('ok')
                        j=10
                        try:
                            i=2
                            while(True):
                                commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                                i=i+1
                                #print(commentaire.text)
                                users.append(commentaire.text)
                                j=j+10

                        except:
                            while(True):
                                try:
                                        page = driver.find_element_by_link_text('View more comments…')
                                        page.click()
                                        #print('ok')
                                        i=2
                                        while(True):
                                            commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                                            i=i+1
                                            #print(commentaire.text)
                                            users.append(commentaire.text)
                                            j=j+10
                                except NoSuchElementException:
                                    done=True
                                    break

    except:
                    err='erreur'
    return(users)


import time 
import pandas as pd

commentaires=[]
cl =[]
links=[]
writer = pd.ExcelWriter('publications2.xlsx', engine='xlsxwriter')
i=1

for lien in liens_publications:
    comments=[]
    users=[]
    link=[]
    driver.get(lien)
    time.sleep(10)
    comments=extract_commentaires()
    time.sleep(10)
    driver.get(lien)
    users = extract_user_names()
    df = pd.DataFrame({'user': users,'commentaire':comments})
    df.to_excel(writer, sheet_name='Pub Num'+str(i))
    i=i+1
    for j in range(len(users)):
        link.append(lien)
    #print(comments)
    #print(users)
    cl = cl + users
    commentaires = commentaires + comments
    links=links+link

    
df1 = pd.DataFrame({'lien publication':links,'user': cl,'commentaire':commentaires})
df1.to_excel(writer, sheet_name='All')
writer.save()

In [5]:
writer.close()

C:\Users\Dream\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [20]:
#Corrections
from selenium.common.exceptions import NoSuchElementException
def extract_commentaires():
    i=1
    comments=[]
    done=False
    try:
        while(True):
                try:
                    commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                    i=i+1
                    #print(commentaire.text)
                    comments.append(commentaire.text)
                except:
                    while(True):
                        page = driver.find_element_by_link_text('View more comments…')
                        page.click()
                        #print('ok')
                        j=10
                        try:
                            i=2
                            while(True):
                                commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                                i=i+1
                                #print(commentaire.text)
                                comments.append(commentaire.text)
                                j=j+10

                        except:
                            while(True):
                                try:
                                        page = driver.find_element_by_link_text('View more comments…')
                                        page.click()
                                        #print('ok')
                                        i=2
                                        while(True):
                                            commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/div[1]')
                                            i=i+1
                                            #print(commentaire.text)    
                                            comments.append(commentaire.text)
                                        j=j+10
                                except NoSuchElementException:
                                    done=True
                                    break

    except:
                    print('erreur')
    return(comments)
driver.get('https://mbasic.facebook.com/story.php?story_fbid=1750414488477444&id=421001574752082&refid=17&_ft_=mf_story_key.1750414488477444%3Atop_level_post_id.1750414488477444%3Atl_objid.1750414488477444%3Acontent_owner_id_new.421001574752082%3Athrowback_story_fbid.1750414488477444%3Apage_id.421001574752082%3Astory_location.4%3Astory_attachment_style.photo%3Aott.AX_5EAqMM6g3uW0v%3Atds_flgs.3%3Athid.421001574752082%3A306061129499414%3A2%3A0%3A1633071599%3A4631501828048357028%3A%3A&__tn__=%2AW-R')
res=extract_commentaires()
print(len(res))

users=[]
def extract_user_names():
    i=1
    comments=[]
    done=False
    try:
        while(True):
                try:
                    commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                    i=i+1
                    #print(commentaire.text)
                    users.append(commentaire.text)
                except:
                    while(True):
                        page = driver.find_element_by_link_text('View more comments…')
                        page.click()
                        #print('ok')
                        j=10
                        try:
                            i=2
                            while(True):
                                commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                                i=i+1
                                #print(commentaire.text)
                                users.append(commentaire.text)
                                j=j+10

                        except:
                            while(True):
                                try:
                                        page = driver.find_element_by_link_text('View more comments…')
                                        page.click()
                                        #print('ok')
                                        i=2
                                        while(True):
                                            commentaire = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div['+str(i)+']/div/h3/a')
                                            i=i+1
                                            #print(commentaire.text)
                                            users.append(commentaire.text)
                                            j=j+10
                                except NoSuchElementException:
                                    done=True
                                    break

    except:
                    print('erreur')
    return(users)

driver.get('https://mbasic.facebook.com/story.php?story_fbid=1750414488477444&id=421001574752082&refid=17&_ft_=mf_story_key.1750414488477444%3Atop_level_post_id.1750414488477444%3Atl_objid.1750414488477444%3Acontent_owner_id_new.421001574752082%3Athrowback_story_fbid.1750414488477444%3Apage_id.421001574752082%3Astory_location.4%3Astory_attachment_style.photo%3Aott.AX_5EAqMM6g3uW0v%3Atds_flgs.3%3Athid.421001574752082%3A306061129499414%3A2%3A0%3A1633071599%3A4631501828048357028%3A%3A&__tn__=%2AW-R')
res=extract_user_names()
print(len(res))



erreur
50
erreur
50


In [11]:
salaire = ["chahriya","double salaire","chahriya","chahreya","chheri","chahryet","شهريتي","شهرية","شهري"]
credit = ["9ardh","9arth","9ardhi","Crédit","credit","كريدي","القرض","قرض","قرضي","الشيكات"]
carte = ["البطاقة البنكية","البطاقة","Carte Banquaire","carte","الكارطة","STBNet"]
service = ["الموزع الآلي","les queues","التليفون","téléphone","service","saff"," queues ","الصف"," استفسار","distributeur ","تحويل الأموال","transfert de compte"]

def est_credit(c):
    credit = ["9ardh","9arth","9ardhi","Crédit","credit","كريدي","القرض","قرض","قرضي","الشيكات"]
    res=False
    for p in credit:
        if(type(c)==list):
            break
        if(c.find(p)!=-1):
            res=True
            break
    return(res)
    
def est_salaire(c):
    salaire = ["فلوسنا","chahriya","double salaire","chahriya","chahreya","chheri","chahryet","شهريتي","شهرية","شهري"]
    res=False
    for p in salaire:
        if(type(c)==list):
            break
        if(c.find(p)!=-1):
            res=True
            break
    return(res)


def est_carte(c):
    carte = ["البطاقة البنكية","البطاقة","Carte Banquaire","carte","الكارطة","STBNet"]
    res=False
    for p in carte:
        if(type(c)==list):
            break
        if(c.find(p)!=-1):
            res=True
            break
    return(res)


def est_service(c):
    service = ["الموزع الآلي","les queues","التليفون","téléphone","service","saff"," queues ","الصف"," استفسار","distributeur ","تحويل الأموال","transfert de compte"]
    res=False
    for p in service:
        if(type(c)==list):
            break
        if(c.find(p)!=-1):
            res=True
            break
    return(res)
#EXCEL N2 
produit=[]
publication=[]
comments=[]

Crédit=[]
Salaire=[]
Carte=[]
Service =[]

for c in commentaires:
    if(est_credit(c)):
        Crédit.append("V")
        produit.append("Crédit")
        publication.append(links[commentaires.index(c)])
        comments.append(c)
    else:
        Crédit.append("F")
    if(est_salaire(c)==True):
        Salaire.append("V")
        produit.append("Salaire")
        publication.append(links[commentaires.index(c)])
        comments.append(c)
    else:
        Salaire.append("F")
    if(est_carte(c)==True):
        Carte.append("V")
        produit.append("Carte")
        publication.append(links[commentaires.index(c)])
        comments.append(c)
    else:
        Carte.append("F")
    if(est_service(c)==True):
        Service.append("V")
        produit.append("Autre")
        publication.append(links[commentaires.index(c)])
        comments.append(c)
    else:
        Service.append("F")
    

    
import pandas as pd
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'user': cl,'lien publication':links,'commentaire':commentaires, 'Crédit': Crédit , 'Salaire': Salaire ,'Carte': Carte , 'Autres Services': Service})
df1= pd.DataFrame({'lien publication':publication,'commentaire':comments,'Type de Produit':produit})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('All3.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')
df1.to_excel(writer, sheet_name='Sheet2')
# Close the Pandas Excel writer and output the Excel file.
writer.save()
